In [17]:
# Import Libraries
import pandas as pd
import spacy
import re
from IPython.display import clear_output

nlp = spacy.load("en_core_web_sm")

In [16]:
## Loading data from csv file
data = pd.read_csv('coupons_ner.csv', names = ['OfferDetails'])
data.head()

,OfferDetails
0,Save $2.00 ONE Downy Liquid Fabric Conditioner...
1,Save $2.00 ONE Tide PODS OR Tide Power PODS (e...
2,Save $2.00 ONE Tide Laundry Detergent (exclude...
3,SAVE $1.00 ON TWO when you buy TWO BOXES (8.9 ...
4,$3.00 OFF when you purchase any THREE (3) Pepp...


In [18]:
## Drop duplicate
print (data.shape)
data = data.drop_duplicates()
print ("Data shape after removing Duplicates.")
data.shape


(886, 1)
Data shape after removing Duplicates.


(788, 1)

In [6]:
## Common word not in products
floatregex = '[+-]?[0-9]+\.[0-9]+'  

non_Product_Words = ['ONE', 'One','TWO', 'THREE', 'FOUR', 'FIVE','SIX', 'SAVE', 'Save', 'save', 'BOXES', 'boxes', 
                   'EXCLUDES','GET', 'excludes', 'travel', 'includes', 'Includes', 'size', 'sizes', 'SIZE',
                   'LARGE', 'small', 'SMALL', 'Limit', 'on','OR', 'of', 'ON','On', 'FREE', 'trial', 'Excluding'
                   'Only', 'ONLY', 'Get', 'SIZE', 'ld', 'oz', 'OZ', 'ct', 'when', 'When', 'below','Purchase','purchase',
                   'Sizes', 'sizes','you', 'You', 'Your', 'larger', 'buy','Small', 'any', 'Any', 'TO',
                   'AND', 'PK', 'pk', 'ml', '(', ')', '$', '¢', '1', '2', '3', '4', '5','\"','...']

product_Words = ['Detergent', 'Laundry', 'Laundry', 'PODS', 'Cheese', 'Liquid']

special_cases = ['or' ,'and','/',',','.', '-']

In [7]:
import winsound
def beep():
    frequency = 2500  # Set Frequency To 2500 Hertz
    duration = 1000  # Set Duration To 1000 ms == 1 second
    winsound.Beep(frequency, duration)

In [8]:
data['OfferDetails'][94]

'SAVE $1.00 TWO Old Spice Anti-Perspirant/Deodorant, Body Wash OR Bar Soap (excludes twin packs, High Endurance, Body Spray and Invisible Spray and trial/travel size).'

In [14]:
## Create training for spacy
import csv

def productLabeling():
    '''
        This function insert the value into the train file.
        O for outside of product name
        B for begining of product name
        I for Inside the product name
    '''
    for i in range(0, data.shape[0]):
        print (i)
        j = 0
        sentence = nlp(data.OfferDetails.iloc[i])
        print (sentence)
        previous = 'A'

        with open('train.tsv', 'a') as out_file:
            tsv_writer = csv.writer(out_file, delimiter='\t', lineterminator='\n')
            for token in sentence:
                j += 1
                if (j % 20) == 0:
                    clear_output()
                    print (i, "Sentence:",sentence)
                if str(token) in non_Product_Words or re.search(floatregex, str(token)):
                    previous = 'O'
                    tsv_writer.writerow([token, 'O'])
                elif ((str(token) in special_cases) and previous== 'O'):
                    tsv_writer.writerow([token, 'O'])
                elif str(token) in product_Words:
                    tsv_writer.writerow([token, 'I'])
                else:
                    print (token)
                    choice = input('Is the word is O (Outer), B(Begining), I (Intermediate): ')
                    if choice != 'O' and choice != 'B' and choice != 'I':
                        beep()
                        choice = input("Please enter proper choice:")
                    if previous == 'O' and choice =='I':
                        choice = 'B'
                    previous = choice
                    tsv_writer.writerow([token, choice])

            tsv_writer.writerow('')
            clear_output(True)
            
productLabeling():

787
SAVE $2.00 when you spend $6 on all Suave® products
spend
Is the word is O (Outer), B(Begining), I (Intermediate): O
6
Is the word is O (Outer), B(Begining), I (Intermediate): O
all
Is the word is O (Outer), B(Begining), I (Intermediate): O
Suave
Is the word is O (Outer), B(Begining), I (Intermediate): B
®
Is the word is O (Outer), B(Begining), I (Intermediate): I
products
Is the word is O (Outer), B(Begining), I (Intermediate): I


In [23]:
train_data = pd.read_csv('train.tsv', sep='\t', encoding='unicode_escape')
train_data.head(30)

,Save,O
0,$,O
1,2.00,O
2,ONE,O
3,Downy,B
4,Liquid,I
5,Fabric,I
6,Conditioner,I
7,72,O
8,ld,O
9,or,O
